In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np

model = YOLO('runs/segment/train/weights/best.pt')

In [ ]:
img = cv2.imread("C:/Users/leva/Desktop/inst_segm/P1/77.jpg")

results = model(img, imgsz=640, iou=0.8, conf=0.19)

In [ ]:
plt.imshow(results[0].masks.data[1].cpu().numpy(), 'gray');

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming results[0].masks.data contains multiple masks
num_masks = results[0].masks.data.shape[0]  # Get the number of masks
cols = 3  # Number of columns for the subplot
rows = (num_masks + cols - 1) // cols  # Calculate number of rows needed

plt.figure(figsize=(15, 5 * rows))  # Adjust figure size based on number of rows

for i in range(num_masks):
    plt.subplot(rows, cols, i + 1)  # Create a subplot for each mask
    plt.imshow(results[0].masks.data[i].cpu().numpy(), cmap='gray')  # Display the mask
    plt.axis('off')  # Optionally turn off axis

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()  # Render all images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Предположим, что у вас есть несколько масок в results[0].masks.data
masks = results[0].masks.data.cpu().numpy()  # Получаем маски и переводим на CPU
num_masks = masks.shape[0]  # Количество масок

# Создаем пустое изображение для наложения масок
combined_image = np.zeros((masks.shape[1], masks.shape[2], 3), dtype=np.uint8)  # RGB изображение

# Определяем цвет для каждой маски (можно настроить по желанию)
colors = plt.cm.jet(np.linspace(0, 1, num_masks))[:, :3]  # Получаем цвета из колор-карты

# Накладываем каждую маску на комбинированное изображение
for i in range(num_masks):
    mask = masks[i]
    color = (colors[i] * 255).astype(np.uint8)  # Преобразуем цвет в формат uint8
    combined_image[mask > 0] = color  # Накладываем цвет на область маски

# Отображаем итоговое изображение
plt.figure(figsize=(10, 10))
plt.imshow(combined_image)
plt.axis('off')  # Отключаем оси
plt.title('Combined Masks')
plt.show()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Assuming 'results' and 'img' are defined earlier in your code
classes = results[0].boxes.cls.cpu().numpy()
class_names = results[0].names

masks = results[0].masks.data
num_masks = masks.shape[0]

# Generate random colors for each mask
colors = [tuple(np.random.randint(0, 256, 3).tolist()) for _ in range(num_masks)]

# Create an overlay for the masks and a labeled image copy
mask_overlay = np.zeros_like(img)
labeled_image = img.copy()

for i in range(num_masks):
    color = colors[i]
    mask = masks[i].cpu()

    # Resize the mask to match the original image size
    mask_resized = cv2.resize(np.array(mask), (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

    class_index = int(classes[i])
    class_name = class_names[class_index]

    # Find contours from the resized mask
    mask_contours, _ = cv2.findContours(mask_resized.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw contours on the labeled image
    cv2.drawContours(labeled_image, mask_contours, -1, color, 5)

    # Put text on the labeled image at the mean position of the contour
    if len(mask_contours) > 0:
        text_position = (int(mask_contours[0][:, 0, 0].mean()), int(mask_contours[0][:, 0, 1].mean()))
        cv2.putText(labeled_image, class_name, text_position, cv2.FONT_HERSHEY_COMPLEX, 1, color, 2)

# Display the labeled image
plt.figure(figsize=(8, 8), dpi=150)
labeled_image = cv2.cvtColor(labeled_image, cv2.COLOR_BGR2RGB)
plt.imshow(labeled_image)
plt.axis('off')
plt.show()

In [ ]:
import cv2
from ultralytics import YOLO
import random
import numpy as np
import sys, os


def process_video_with_tracking(model, input_video_path, show_video=True, save_video=False, output_video_path="output_video.mp4"):
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        raise Exception("Error: Could not open video file.")

    # Get input video frame rate and dimensions
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the output video writer
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps/2, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model.track(frame, iou=0.6, conf=0.75, persist=True, imgsz=640, verbose=False, tracker="botsort.yaml")

        if results[0].boxes.id != None: # this will ensure that id is not None -> exist tracks
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)

            for box, id in zip(boxes, ids):
                # Generate a random color for each object based on its ID
                random.seed(int(id))
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3],), color, 4)
                cv2.putText(
                    frame,
                    f"Id {id}",
                    (box[0], box[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.5,
                    (0, 255, 255),
                    2,
                )

        if save_video:
            out.write(frame)

        if show_video:
            frame = cv2.resize(frame, (frame.shape[1]//2, frame.shape[0]//2))
            cv2.imshow("frame", frame)

        if cv2.waitKey(int(1000/fps)) & 0xFF == ord("q"):
            break

    # Release the input video capture and output video writer
    cap.release()
    if save_video:
        out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()


# Example usage:
model = YOLO('runs/segment/train/weights/best.pt')
process_video_with_tracking(model, input_video_path="video1_2.mp4",
                                          show_video=True, save_video=False,
                                          output_video_path="result_videos/output_video.mp4")
